In [2]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab,ocr,preprocessing,file-conversion,pdf]


In [3]:
from haystack.telemetry import tutorial_running

tutorial_running(8)


In [4]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [5]:
from haystack.utils import fetch_archive_from_http


# This fetches some sample files to work with
doc_dir = "data/tutorial8"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/preprocessing_tutorial8.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)


INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/preprocessing_tutorial8.zip to 'data/tutorial8'


True

In [6]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor


converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="data/tutorial8/classics.txt", meta=None)[0]

converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_pdf = converter.convert(file_path="data/tutorial8/bert.pdf", meta=None)[0]

converter = DocxToTextConverter(remove_numeric_tables=False, valid_languages=["en"])
doc_docx = converter.convert(file_path="data/tutorial8/heavy_metal.docx", meta=None)[0]


In [7]:
from haystack.utils import convert_files_to_docs


all_docs = convert_files_to_docs(dir_path=doc_dir)


INFO:haystack.utils.preprocessing:Converting data/tutorial8/classics.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial8/bert.pdf
INFO:haystack.utils.preprocessing:Converting data/tutorial8/heavy_metal.docx


In [8]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 38.16docs/s]

n_docs_input: 1
n_docs_output: 51


In [9]:
# Not respecting sentence boundary vs respecting sentence boundary

preprocessor_nrsb = PreProcessor(split_respect_sentence_boundary=False)
docs_nrsb = preprocessor_nrsb.process([doc_txt])

print("RESPECTING SENTENCE BOUNDARY")
end_text = docs_default[0].content[-50:]
print('End of document: "...' + end_text + '"')
print()
print("NOT RESPECTING SENTENCE BOUNDARY")
end_text_nrsb = docs_nrsb[0].content[-50:]
print('End of document: "...' + end_text_nrsb + '"')


Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 54.89docs/s]

RESPECTING SENTENCE BOUNDARY
End of document: "...rnerstone of a typical elite European education.

"

NOT RESPECTING SENTENCE BOUNDARY
End of document: "...xts used as part of a curriculum, both derive from"


In [10]:
# Sliding window approach

preprocessor_sliding_window = PreProcessor(split_overlap=3, split_length=10, split_respect_sentence_boundary=False)
docs_sliding_window = preprocessor_sliding_window.process([doc_txt])

doc1 = docs_sliding_window[0].content[:200]
doc2 = docs_sliding_window[1].content[:100]
doc3 = docs_sliding_window[2].content[:100]

print('Document 1: "' + doc1 + '..."')
print('Document 2: "' + doc2 + '..."')
print('Document 3: "' + doc3 + '..."')


Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 21.46docs/s]

Document 1: "Classics or classical studies is the study of classical antiquity,..."
Document 2: "of classical antiquity, and in the Western world traditionally refers..."
Document 3: "world traditionally refers to the study of Classical Greek and..."


In [11]:
all_docs = convert_files_to_docs(dir_path=doc_dir)
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")


INFO:haystack.utils.preprocessing:Converting data/tutorial8/classics.txt
INFO:haystack.utils.preprocessing:Converting data/tutorial8/bert.pdf
INFO:haystack.utils.preprocessing:Converting data/tutorial8/heavy_metal.docx
Preprocessing: 100%|██████████| 3/3 [00:00<00:00, 38.88docs/s]

n_files_input: 3
n_docs_output: 175
